In [34]:
import numpy
import random
import matplotlib.pyplot as plt
import math

#Declarando variables
inventario = 60          #Estado inicial
meses = (120 * 30)         #Meses a ejecutar
dias = 0
tiempo = 0
cont_clien = 0
producto = 0
dia_pedido = 30 #Es el dia en el que se hace el pedido
pedido = 0 #Cantidad de productos que se pediran al proveedor
cont_inventario = 1
cont = 0
indice = 0
s_menor = [20,20,20,20,40,40,40,60,60] # Politicas para el menor numero de productos que podemos tener
s_mayor = [40,60,80,100,60,80,100,80,100]# Politicas para el maximo numero de productos que podemos tener
lista_costo = [] #Aqui se guardan los costo de orden de cada mes 
costo_orden = 0 #Guarda el costo de cada orden en el mes
bandera = False
tProveedor = 0 #Guarda la llegada del proveedor
lista_escases = [] #Guarda los costos de escases
lista_mantenimiento = [] #Guarda los costos de mantenimiento

lista_espera = [] #Si no hay productos en el almacen aqui se van formando a los clientes
diasProveedor = 0 #Tiempo en que tardará el porveedor en surtir el inventario


####Variable para guardar promedios
promedio_Orden = 0
promedio_Escaces = 0
promedio_Mantenimiento = 0
promedio_Total = 0

promedio_Mes = []


#FUNCIONES
#Funcion que genera num aleatorio en dias (Funcion de distribucion exponencial)
def llegada_cliente():
    aleatorio = numpy.random.uniform(0.0,1.0)                  #Genera numero aleatorio
    dia_llegada = math.ceil((math.log(1 - aleatorio)/-10)*30) #Calcula tiempo en dias, (-0.1 representa promedio llegada por cliente), se multimpla por 30 para convertirlo a dias
    #math.ceil(dia_llegada)
    return dia_llegada 

#Funcion que genera num aleatorio para compra de prodcuto de 1 a 4 (Funcion distribucion acumulada)
def productoCompra():
    numProductos = numpy.random.uniform(0.0, 1.0)               #Aleatorio para determinar el numero de productos
    
    if(numProductos < 0.16):                                    
        return 1
    
    if(numProductos >= 0.16 and numProductos < 0.5):
        return 2
    
    if(numProductos >= 0.5 and numProductos < 0.83):
        return 3
    
    if(numProductos >= 0.83 and numProductos <= 1):
        return 4
    
#Calcula los costos de escases y mantenimiento, dependiendo del valor que tenga el inventario
def mantenimiento(dias):
    global inventario, lista_escases, lista_mantenimiento
    ce = 0 #Guarda cada uno de los costos de escases
    cm = 0 #Guarda cada uno de los costos de mantenimiento
    if inventario < 0:                          #Si no hay productos en inventario, genera calculos 
        ce = 5 * abs(inventario) * dias       
        lista_escases.append(ce)
        
    else:
        cm = 1 * inventario * dias
        lista_mantenimiento.append(cm)


#Funcion encargada de calcular los promedios necesarios
def calcula_promedios():
    global promedio_Escaces, promedio_Mantenimiento, promedio_Orden
    
    promedio_Escaces = numpy.sum(lista_escases)/ cont_clien
    promedio_Mantenimiento = numpy.sum(lista_mantenimiento) / cont_clien    
    promedio_Orden = numpy.sum(lista_costo) / 120    
    

def reiniciaVariables():
    global promedio_Escaces, promedio_Mantenimiento, promedio_Orden,promedio_Total,tiempo
    global dias, cont_clien, producto, pedido, cont_inventario, indice, lista_costo, costo_orden, tProveedor
    global lista_escases, lista_mantenimiento, lista_espera, diasProveedor
    promedio_Escaces = 0
    promedio_Mantenimiento = 0
    promedio_Orden = 0
    promedio_Total = 0
    tiempo = 0
    
    dias = 0
    cont_clien = 0
    producto = 0
    pedido = 0 #Cantidad de productos que se pediran al proveedor
    cont_inventario = 1
    
    indice = 0
    lista_costo = [] 
    costo_orden = 0 

    tProveedor = 0 
    lista_escases = [] 
    lista_mantenimiento = [] 

    lista_espera = [] 
    diasProveedor = 0 
    dia_pedido = 30
    

    
#Verifica en el inventario la cantidad de productos para vender
def hay_productos(producto):
    global inventario, lista_espera
  
    if inventario <= 0:                  #Cuando no hay productos en inventario
       
        lista_espera.append(producto)                       #Mantiene en lista de espera la cantidad de productos pendientes a entregar
        inventario = inventario - producto                  #Actualiza inventario (en numeros negativos por estar en deuda)

    else:                                #Cuando si hay productos en inventario 
        if inventario <= producto:                          #Pero no alcanza a sutir por completo la cantidad de productos vendidos
            inventario = inventario - producto              #Se descuentan los productos de inventario y los faltantes quedan pendientes de entregar (en numeros negativos por estar en deuda)  
            producto = abs(inventario)                      #Producto pendiente de surtir al cliente
            lista_espera.append(producto)                   #mantiene en lista de espera la cantidad de productos pendientes a entrega
            #print("ADEUDO  Inventario ", inventario, "  Debo a cliente ", producto)
        else:                                               #Cuando existen los productos necesarios en invnetario para surtir la compra al cliente
            inventario = inventario - producto              #Se descuentan de inventario

#Calcula la cantidad de productos y el costo para solicitar a proveedor y surtir el inventario 
def pedidoInventario(cont):
    global pedido, s_mayor, costo_orden, lista_costo, inventario
    
    
    pedido = s_mayor[indice] - inventario                   #Diferencia del num maximo de productos en inventario - los productos actuales de inventario
    costo_orden = 32 + (3 * pedido)                         #Calculando el costo para ordenar pedido a distribuidor
    lista_costo.append(costo_orden)                         #Almacenando el valor del costo de orden en lista(lo alamcena por cada mes)
    tiempoProveedor()
    
    
#Calcula los dias (entre 15 y 30) en que tardará en distribuir los productos el proveedor
def tiempoProveedor():
    global bandera, diasProveedor
    diasProveedor = numpy.random.uniform(0.5,1.0) * 30                 #Genera numero aleatorio entre 0.5 y 1, se multimplica por 30 para generar dias entre 15 y 30
    bandera = True

    



dias = llegada_cliente()                                    #Calculando la llegada del futuro cliente en dias
#print("Tiempo ", tiempo, "  Cliente ", cont_clien , "  Productos ", producto, "  Inventario ", inventario ,"   T. prox llegada ", dias, )
tiempo = dias + tiempo                                      #Actualizando el reloj al tiempo actual mas los dias del la proxima llegada del futuro cliente

print("Politica                      Costo Total                     Costo de Orden                       Costo de mantenimiento             Costo de escases")
for cont in range(len(s_menor)):
    inventario = s_mayor[cont]
#     print("******* ITERACION NUMERO ", (cont+1), "********")

    
    
    while tiempo <= meses:
        
        
        cont_clien += 1                                                #Contador de clientes
        producto = productoCompra()                                    #Generando cantidad de productos a comprar                                
        hay_productos(producto)                                        #Verificando en inventario si existe disponibilidad de productos
        
        dias = llegada_cliente()                                       #Calculando la llegada del futuro cliente en dias
#         print("Tiempo ", tiempo, "  Cliente ", cont_clien , "  Productos ", producto, "  Inventario ", inventario ,"   T. prox llegada ", (dias+tiempo) )
        tiempo = dias + tiempo                                          #Actualizando el reloj al tiempo actual mas los dias del la proxima llegada del futuro cliente
        
        if tiempo >= ((dia_pedido * cont_inventario)+1) and tiempo < (tiempo + 1):       #Cuando es inicio de mes
            mantenimiento(dias) #Se calculan el cosoto de mantenimiento o escases dependiendo de los valores del inventario
            if inventario < s_menor[indice]:                           #Si inventario tiene menos productos que S menor, entonces realizará pedido
                pedidoInventario(cont)                                      #Se solicita surtir el inventario y generar costo
                tProveedor = (dia_pedido * cont_inventario) + diasProveedor
#             print("PEDIDO DE MES ", cont_inventario, "----> Num productos pedidos ", pedido, "   Costo ", costo_orden, "   En Inventario hay ", inventario, "  Dias que tardara en surtir ", diasProveedor, "  Dia que llega proveedor ", tProveedor)  
            cont_inventario += 1

        if bandera:
            if tiempo > tProveedor:
                inventario = pedido                                    #Actualizando el inventario
#                 print("LLEGA PROVEDOR EN TIEMPO ", tProveedor, "  DISTRIBUYE ", pedido, " Inventario hay ", inventario)
                tamañoList = len(lista_espera)
                lista_espera.sort()
               
                for numProduct in range(tamañoList):
                    numProd = lista_espera.pop()
                    inventario -= numProd
#                     print("Surtiendo prod pendiente al cliente ", numProduct+1, " la cantidad de ", numProd, " inventario ", inventario)
                    
                bandera = False
    indice += 1
    
    #print("Lista de costos por pedido ", lista_costo)
    #print("Lista de costos por escaces ", lista_escases)
    #print("Lista de costos por mantenimiento ", lista_mantenimiento)
    
    calcula_promedios()
    promedio_Total = (abs(promedio_Escaces) + promedio_Mantenimiento + promedio_Orden)
    print(" (",s_menor[cont],",",s_mayor[cont],")                     ", round(promedio_Total,2),"                         ",round(promedio_Orden,2),"                              ",round(promedio_Mantenimiento,2),"                                ",round(promedio_Escaces,2))
    reiniciaVariables()

Politica                      Costo Total                     Costo de Orden                       Costo de mantenimiento             Costo de escases
 ( 20 , 40 )                      116.24                           100.45                                8.18                                  7.61
 ( 20 , 60 )                      126.6                           107.82                                7.86                                  10.93
 ( 20 , 80 )                      127.57                           106.96                                7.42                                  13.2
 ( 20 , 100 )                      127.98                           100.95                                10.67                                  16.36
 ( 40 , 60 )                      135.98                           112.04                                7.13                                  16.8
 ( 40 , 80 )                      122.31                           102.68                                8